# Catlab.jl is a package in Julia that implements Category Theory.

⊣ this symbol is "\vdash" 

⋅ this symbol is "\cdot"

→ this symbol is "\to"

In [7]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAp/Mathematical-Short-Notes/Fields/Category-Theory/notebooks`


## 1. Introduction to Catlab

Below let's use Catlab in order to define a Category. This might seem odd, since Catlab is already Category Theory for Julia,
so why are we defining a Category? Well, first, this is already implemented, so we are doing it only to showcase. But also,
Catlab actually implement Generalized Algebraic Theories, which permits the definition of "objects" other than Cateogories.
Thus, the package is actually more general.

In [20]:
using Catlab

In [22]:
@theory Category{Ob,Hom} begin
  @op begin
    (→) := Hom
    (⋅) := compose
  end

  Ob::TYPE
  Hom(dom::Ob, codom::Ob)::TYPE

  id(A::Ob)::(A → A)
  compose(f::(A → B), g::(B → C))::(A → C) ⊣ (A::Ob, B::Ob, C::Ob)
    

  (f ⋅ g) ⋅ h == f ⋅ (g ⋅ h) ⊣ (A::Ob, B::Ob, C::Ob, D::Ob,
                                f::(A → B), g::(B → C), h::(C → D))
  f ⋅ id(B) == f ⊣ (A::Ob, B::Ob, f::(A → B))
  id(A) ⋅ f == f ⊣ (A::Ob, B::Ob, f::(A → B))
end;

Now that we defined what a category is, let's in fact instantiate a category, i.e.
let's do an example of an actual category. For that, we'll use the category of Finite Vector Spaces,
where the morphisms are matrices (i.e. linear transformations) and the objects are finite vector spaces
(which are all isomorphic to ℝⁿ for n the dimension).

In [37]:
using LinearAlgebra: I

struct MatrixDomain
  eltype::Type
  dim::Int
end

@instance Category{MatrixDomain, Matrix} begin
  dom(M::Matrix) = MatrixDomain(eltype(M), size(M,1))
  codom(M::Matrix) = MatrixDomain(eltype(M), size(M,2))

  id(m::MatrixDomain) = Matrix{m.eltype}(I, m.dim, m.dim)
  compose(M::Matrix, N::Matrix) = M*N
end


A = rand(5,2)
B = rand(2,2)
@show dom(A), codom(A)
id(dom(A)) # The identity morphism is the identity matrix
# compose(B,A) <- not composable
compose(A,B)

(dom(A), codom(A)) = (MatrixDomain(Float64, 5), MatrixDomain(Float64, 2))


5×2 Matrix{Float64}:
 0.598137  0.666098
 0.842528  0.857053
 0.407519  0.502027
 0.506689  0.668944
 0.53532   0.501861

Next, let's implement a more "convoluted" example. Let's use Catlab in order to create a
way to enforce that functions need to match dom/codom in order to be composed.
Note that does not happen naturally in Julia. A function is Julia does not have a parametric
type based on it's domain and codomain. Part of the reason is that most functions in Julia
do not have an output type enforcer (although this can be done, it's seldom used and 
actually desincentivised).

Hence, how can we guarantee that `compose(f,g)` actually can be composed, before evaluating the results?
The answer is with Catlab. But, before we show the example, here is 

In [58]:
struct JFunction
    f::Function
    dom::Type
    codom::Type
end
(::JFunction)(x) = f(x)
ȷf = JFunction(x->x, Int, Int)

JFunction(var"#13#14"(), Int64, Int64)

In [69]:
@instance Category{Type, JFunction} begin
  dom(ȷf::JFunction) = getfield(ȷf,:dom)
  codom(ȷf::JFunction) = getfield(ȷf,:codom)

  id(d::Type) = JFunction(x->x, d, d)
  compose(ȷg::JFunction, ȷf::JFunction) = JFunction(ȷg ∘ ȷf, ȷf.dom, ȷg.codom)
end

In [71]:
ȷf ⋅ ȷf

JFunction(JFunction(var"#13#14"(), Int64, Int64) ∘ JFunction(var"#13#14"(), Int64, Int64), Int64, Int64)